In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import random
import itertools
import gzip
import pyranges as pr

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as colors

from GraphRicciCurvature.OllivierRicci import OllivierRicci

# Load data

In [2]:
%%time
fpath = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/f1219_population_hic/4DNFI9H5B7J4.pairs.gz"
nrows = 1000000

def load_pairs(fpath, nrows=None, usecols=None):
    # discover column names from the '#columns:' line
    cols = None
    with gzip.open(fpath, "rt") as fh:
        for line in fh:
            if line.startswith("#columns"):
                cols = line.split(":", 1)[1].strip().split()
                break
            if not line.startswith("#"):
                break
    if cols is None:
        # sensible default if no header is present
        cols = ["readID","chr1","pos1","chr2","pos2","strand1","strand2"]

    if usecols is not None:
        # preserve file order while subsetting
        cols = [c for c in cols if c in usecols]

    df = pd.read_csv(
        fpath,
        sep="\t",
        comment="#",
        compression="gzip",
        header=None,
        names=cols,
        usecols=cols,
        low_memory=False,
        nrows=nrows,
    )
    return df

df = load_pairs(fpath, nrows=nrows)
print(f"{df.shape=}")
df.head()

df.shape=(1000000, 10)
CPU times: user 869 ms, sys: 279 ms, total: 1.15 s
Wall time: 1.16 s


,readID,chrom1,pos1,chrom2,pos2,strand1,strand2,pair_type,mapq1,mapq2
0,81236149,chr1,3000037,chr1,3000363,+,-,UU,14,52
1,53397258,chr1,3000046,chr1,3000430,+,-,UU,9,60
2,31007112,chr1,3000152,chr1,3000336,+,-,UU,24,60
3,4694079,chr1,3000181,chr1,3000644,+,-,UU,31,60
4,175297042,chr1,3000254,chr1,3000563,+,-,UU,52,25


# Load genes

In [3]:
fpath = "../resources/mouse_genes.txt.gz"

gdf = pd.read_csv(fpath, sep='\t', low_memory=False)
gdf.head()

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene start (bp),Gene end (bp),Transcript start (bp),Transcript end (bp),Transcription start site (TSS),Gene name,Gene % GC content,Gene type,Chromosome/scaffold name
0,ENSMUSG00000064336,ENSMUSG00000064336.1,ENSMUST00000082387,ENSMUST00000082387.1,1,68,1,68,1,mt-Tf,30.88,Mt_tRNA,MT
1,ENSMUSG00000064337,ENSMUSG00000064337.1,ENSMUST00000082388,ENSMUST00000082388.1,70,1024,70,1024,70,mt-Rnr1,35.81,Mt_rRNA,MT
2,ENSMUSG00000064338,ENSMUSG00000064338.1,ENSMUST00000082389,ENSMUST00000082389.1,1025,1093,1025,1093,1025,mt-Tv,39.13,Mt_tRNA,MT
3,ENSMUSG00000064339,ENSMUSG00000064339.1,ENSMUST00000082390,ENSMUST00000082390.1,1094,2675,1094,2675,1094,mt-Rnr2,35.40,Mt_rRNA,MT
4,ENSMUSG00000064340,ENSMUSG00000064340.1,ENSMUST00000082391,ENSMUST00000082391.1,2676,2750,2676,2750,2676,mt-Tl1,44.00,Mt_tRNA,MT


# Filter Genes

In [4]:
gdf['']

KeyError: ''

In [ ]:
break

In [ ]:
print(gdf[['Gene name', 'Chromosome/scaffold name', 'Gene start (bp)', 'Gene end (bp)']].head().to_string(index=False))

# Merge and Filter

In [ ]:
%%time
def make_gene_ranges(gdf, chrom="chr1"):
    """Return PyRanges of genes on `chrom` (1-based inclusive -> 0-based half-open)."""
    g = (gdf.rename(columns={
            'Gene name':'gene',
            'Chromosome/scaffold name':'chrom',
            'Gene start (bp)':'start',
            'Gene end (bp)':'end'
        }).copy())
    g['chrom'] = g['chrom'].astype(str).str.replace('^chr','', regex=True).radd('chr')
    g = g[g['chrom'] == chrom]
    return pr.PyRanges(
        g.assign(Start=g['start']-1, End=g['end'])
         [['chrom','Start','End','gene']]
         .rename(columns={'chrom':'Chromosome'}).drop_duplicates()
    )

chrom = 'chr1'
gr_genes = make_gene_ranges(gdf, chrom=chrom)
print(f"{len(gr_genes)=}")
print(gr_genes.df.head().to_string(index=False))

In [ ]:
%%time
def gene_gene_contacts(df, gr_genes, chrom="chr1"):
    """Return contacts on `chrom` where both ends overlap a gene in `gr_genes`."""
    c = df[(df['chrom1'] == chrom) & (df['chrom2'] == chrom)].copy()
    if c.empty:
        return pd.DataFrame(columns=['chrom1','pos1','chrom2','pos2','gene1','gene2'])

    p1 = pr.PyRanges(pd.DataFrame({
        'Chromosome': c['chrom1'], 'Start': c['pos1']-1, 'End': c['pos1'], 'i': c.index
    }))
    p2 = pr.PyRanges(pd.DataFrame({
        'Chromosome': c['chrom2'], 'Start': c['pos2']-1, 'End': c['pos2'], 'i': c.index
    }))

    def end_to_gene(p, label):
        j = p.join(gr_genes).as_df()
        if j.empty:
            return pd.DataFrame(columns=['i', label])
        return j[['i','gene']].rename(columns={'gene': label})

    a1 = end_to_gene(p1, 'gene1')
    a2 = end_to_gene(p2, 'gene2')

    out = (c.assign(i=c.index)
             .merge(a1, on='i', how='inner')
             .merge(a2, on='i', how='inner')
             .drop(columns='i'))
    return out[['chrom1','pos1','chrom2','pos2','gene1','gene2']]

pdf = gene_gene_contacts(df, gr_genes, chrom="chr1")
print(pdf.head().to_string(index=False))

In [ ]:
%%time
X = pdf.groupby(['gene1', 'gene2'])['pos1'].count().reset_index()
X_p = X.copy()
X_p.columns = ['gene2', 'gene1', 'pos1']
X = pd.concat([X, X_p], ignore_index=True)
X = pd.pivot_table(
    X, 
    index='gene1',
    columns='gene2',
    values='pos1',
    fill_value=0.0,
)

print(f"{X.shape=}")
X

In [ ]:
sns.heatmap(np.log1p(X))